In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, f1_score, roc_auc_score
from constants import LABEL_COLS

In [ ]:
f1_scores = []
for cutoff in np.linspace(0, 1, num=50):
    results_df = pd.read_csv('../data/predicted_labels.csv')

    binary_labels = []
    for key in LABEL_COLS:
        binary_key = f'binary_{key}'
        results_df[binary_key] = (results_df[key] >= cutoff).astype(int)

        binary_labels.append(binary_key)
    results_df.drop(LABEL_COLS, axis=1, inplace=True)
    data_df = pd.read_csv('../data/raw/test.csv')

    label_df = pd.read_csv('../data/raw/test_labels.csv')

    test_df = data_df.set_index('id').join(label_df.set_index('id'))
    master_df = test_df.join(results_df.set_index('id'))
    CONDITIONS = [f'{label} != -1' for label in LABEL_COLS]
    QUERY_STRING = ' & '.join(CONDITIONS)
    master_df = master_df.query(QUERY_STRING)
    y_test = master_df[LABEL_COLS]
    predictions = master_df[binary_labels]

    f1_scores.append((cutoff, roc_auc_score(y_test, predictions, average='macro')))

In [ ]:
cutoff, _ = max(f1_scores, key=lambda x: x[1])
results_df = pd.read_csv('../data/predicted_labels.csv')

binary_labels = []
for key in LABEL_COLS:
    binary_key = f'binary_{key}'
    results_df[binary_key] = (results_df[key] >= cutoff).astype(int)

    binary_labels.append(binary_key)
results_df.drop(LABEL_COLS, axis=1, inplace=True)
data_df = pd.read_csv('../data/raw/test.csv')

label_df = pd.read_csv('../data/raw/test_labels.csv')

test_df = data_df.set_index('id').join(label_df.set_index('id'))
master_df = test_df.join(results_df.set_index('id'))
CONDITIONS = [f'{label} != -1' for label in LABEL_COLS]
QUERY_STRING = ' & '.join(CONDITIONS)
master_df = master_df.query(QUERY_STRING)
y_test = master_df[LABEL_COLS]
predictions = master_df[binary_labels]

print(roc_auc_score(y_test, predictions))